In [1]:
import duckdb
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tpch").getOrCreate()


25/07/14 23:35:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
all_configs = spark.sparkContext.getConf().getAll()

In [3]:
configs = {}
for config in all_configs:
    configs[config[0]] = config[1]

In [4]:
configs.get('spark.sql.catalog.demo.type')

'rest'

In [5]:
configs.keys()

dict_keys(['spark.app.submitTime', 'spark.eventLog.enabled', 'spark.driver.host', 'spark.history.fs.logDirectory', 'spark.sql.warehouse.dir', 'spark.sql.catalog.demo.s3.endpoint', 'spark.eventLog.dir', 'spark.app.id', 'spark.serializer.objectStreamReset', 'spark.master', 'spark.submit.deployMode', 'spark.app.startTime', 'spark.driver.extraJavaOptions', 'spark.sql.catalogImplementation', 'spark.sql.catalog.demo.warehouse', 'spark.sql.catalog.demo.io-impl', 'spark.driver.port', 'spark.executor.id', 'spark.app.name', 'spark.sql.extensions', 'spark.sql.catalog.demo.uri', 'spark.sql.catalog.demo.type', 'spark.rdd.compress', 'spark.executor.extraJavaOptions', 'spark.sql.catalog.demo', 'spark.sql.defaultCatalog', 'spark.submit.pyFiles', 'spark.ui.showConsoleProgress'])

In [6]:
spark.sql('create database if not exists bronze')
spark.sql('create database if not exists silver')
spark.sql('create database if not exists gold')

DataFrame[]

In [8]:
# create TPCH database 
# TABLES
customer = 'select * from customer'
lineitem = 'select * from lineitem'
nation = 'select * from nation'
part = 'select * from part'
partsupp = 'select * from partsupp'
region = 'select * from region'
supplier = 'select * from supplier'

def load_tpch():
    con = duckdb.connect()
    con.execute('INSTALL tpch; LOAD tpch;CALL dbgen(sf = 0.01);')
    
    # customer table
    customer_df = spark.createDataFrame(con.sql(customer).df())
    customer_df.write.mode('overwrite').saveAsTable('bronze.customer')

    # lineitem
    lineitem_df = spark.createDataFrame(con.sql(lineitem).df())
    lineitem_df.write.mode('overwrite').saveAsTable('bronze.lineitem')

    # part table
    part_df = spark.createDataFrame(con.sql(part).df())
    part_df.write.mode('overwrite').saveAsTable('bronze.part')

    # nation
    nation_df = spark.createDataFrame(con.sql(nation).df())
    nation_df.write.mode('overwrite').saveAsTable('bronze.nation')

    # partsupp table
    partsupp_df = spark.createDataFrame(con.sql(partsupp).df())
    partsupp_df.write.mode('overwrite').saveAsTable('bronze.partsupp')

    # region
    region_df = spark.createDataFrame(con.sql(region).df())
    region_df.write.mode('overwrite').saveAsTable('bronze.region')

    # supplier
    supplier_df = spark.createDataFrame(con.sql(supplier).df())
    supplier_df.write.mode('overwrite').saveAsTable('bronze.supplier')

    con.commit()
    con.close()


In [9]:
load_tpch()

25/07/14 23:35:55 WARN TaskSetManager: Stage 1 contains a task of very large size (1061 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

In [11]:
%%sql
use bronze;

++
||
++
++

In [13]:
%%sql 
show tables;

namespace,tableName,isTemporary
bronze,customer,False
bronze,lineitem,False
bronze,nation,False
bronze,part,False
bronze,partsupp,False
bronze,region,False
bronze,supplier,False


In [14]:
%%sql 
select * from customer limit 10;

c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
1,Customer#000000001,j5JsirBM9PsCy0O1m,15,25-989-741-2988,711.56,BUILDING,y final requests wake slyly quickly special accounts. blithely
2,Customer#000000002,487LW1dovn6Q4dMVymKwwLE9OKf3QG,13,23-768-687-3665,121.65,AUTOMOBILE,y carefully regular foxes. slyly regular requests about the bli
3,Customer#000000003,fkRGN8nY4pkE,1,11-719-748-3364,7498.12,AUTOMOBILE,fully. carefully silent instructions sleep alongside of the slyly regular asymptotes. quickly regular
4,Customer#000000004,4u58h fqkyE,4,14-128-190-5944,2866.83,MACHINERY,sublate. fluffily even instructions are about th
5,Customer#000000005,hwBtxkoBF qSW4KrIk5U 2B1AU7H,3,13-750-942-6364,794.47,HOUSEHOLD,equests haggle furiously against the pending packa
6,Customer#000000006,"g1s,pzDenUEBW3O,2 pxu0f9n2g64rJrt5E",20,30-114-968-4951,7638.57,AUTOMOBILE,quickly silent asymptotes are slyly regular excuses. instructions wake furiously? quickly bold courts p
7,Customer#000000007,8OkMVLQ1dK6Mbu6WG9 w4pLGQ n7MQ,18,28-190-982-9759,9561.95,AUTOMOBILE,"ounts. ironic, regular accounts sleep. final requests haggle quickly after the"
8,Customer#000000008,"j,pZ,Qp,qtFEo0r0c 92qobZtlhSuOqbE4JGV",17,27-147-574-9335,6819.74,BUILDING,riously final excuses sublate quickly among the fluffily even foxes. quickly final packages haggle furiously furi
9,Customer#000000009,vgIql8H6zoyuLMFNdAMLyE7 H9,8,18-338-906-3675,8324.07,FURNITURE,ss pinto beans believe slyly quiet deposits-- doggedly bold packages boost. quickly ironic de
10,Customer#000000010,"Vf mQ6Ug9Ucf5OKGYq fsaX AtfsO7,rwY",5,15-741-346-9870,2753.54,HOUSEHOLD,g quickly after the evenly bold


In [15]:
%%sql
describe bronze.customer;

col_name,data_type,comment
c_custkey,bigint,None
c_name,string,None
c_address,string,None
c_nationkey,bigint,None
c_phone,string,None
c_acctbal,double,None
c_mktsegment,string,None
c_comment,string,None


In [16]:
%%sql
show create table bronze.customer;

createtab_stmt
"CREATE TABLE demo.bronze.customer ( c_custkey BIGINT, c_name STRING, c_address STRING, c_nationkey BIGINT, c_phone STRING, c_acctbal DOUBLE, c_mktsegment STRING, c_comment STRING)USING icebergLOCATION 's3://warehouse/bronze/customer'TBLPROPERTIES ( 'created-at' = '2025-07-14T23:35:49.635183045Z', 'current-snapshot-id' = '2044138703849656797', 'format' = 'iceberg/parquet', 'format-version' = '2', 'write.format.default' = 'parquet', 'write.parquet.compression-codec' = 'zstd')"
